In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import copy

# 设备配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")


# 1. 多模态数据集类（支持样本索引和标签）
class MultimodalDataset(Dataset):
    def __init__(self, image_data=None, text_data=None, vector_data=None, targets=None, is_test=False, device=device):
        self.image_data = image_data
        self.text_data = text_data
        self.vector_data = vector_data
        self.targets = targets  # 标签（y）
        self.is_test = is_test
        self.device = device
        
        # 确定样本数量
        if image_data is not None:
            self.num_samples = len(image_data)
        elif text_data is not None:
            self.num_samples = len(text_data)
        else:
            self.num_samples = len(vector_data)

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        sample = {'index': idx}  # 样本索引
        if self.image_data is not None:
            sample['image'] = torch.tensor(self.image_data[idx], dtype=torch.float32).to(self.device)
        if self.text_data is not None:
            sample['text'] = torch.tensor(self.text_data[idx], dtype=torch.float32).to(self.device)
        if self.vector_data is not None:
            sample['vector'] = torch.tensor(self.vector_data[idx], dtype=torch.float32).to(self.device)
        if self.targets is not None:
            sample['target'] = torch.tensor(self.targets[idx], dtype=torch.float32).to(self.device)  # 标签
        return sample


# 2. 模态编码器（保持不变）
class ImageEncoder(nn.Module):
    def __init__(self, output_dim=128):
        super().__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3)
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3)
        )
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(3, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3)
        )
        self.spatial_attn = nn.Sequential(
            nn.Conv2d(3, 1, kernel_size=1, stride=1),
            nn.Sigmoid()
        )
        self.fc = nn.Linear(3 * 8 * 8, output_dim)

    def forward(self, x):
        x1 = self.conv_block1(x) + x
        x2 = self.conv_block2(x1) + F.interpolate(x1, size=x1.shape[-1]//2)
        x3 = self.conv_block3(x2) + F.interpolate(x2, size=x2.shape[-1]//2)
        attn = self.spatial_attn(x3)
        x3 = x3 * attn
        x3 = x3.view(x3.size(0), -1)
        return self.fc(x3)


class TextEncoder(nn.Module):
    def __init__(self, input_dim=50, hidden_dim=64, output_dim=128):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=2,
            batch_first=True,
            bidirectional=True,
            dropout=0.2
        )
        self.text_attn = nn.Linear(hidden_dim * 2, 1)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        attn_weights = F.softmax(self.text_attn(out).squeeze(-1), dim=1)
        weighted_out = torch.bmm(attn_weights.unsqueeze(1), out).squeeze(1)
        return self.fc(weighted_out)


class VectorEncoder(nn.Module):
    def __init__(self, input_dim=32, output_dim=128):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        return self.mlp(x)


# 3. 多模态融合模型（保持不变）
class MultimodalEncoder(nn.Module):
    def __init__(self, image_encoder, text_encoder, vector_encoder, latent_dim=128, output_dim=1):
        super().__init__()
        self.image_encoder = image_encoder
        self.text_encoder = text_encoder
        self.vector_encoder = vector_encoder
        self.latent_dim = latent_dim
        
        self.modal_weight = nn.Parameter(torch.ones(3))
        self.cross_attn = nn.MultiheadAttention(embed_dim=128, num_heads=4, batch_first=True)
        
        self.fusion = nn.Sequential(
            nn.Linear(128 * 3, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, latent_dim),
            nn.BatchNorm1d(latent_dim)
        )
        
        self.regressor = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )
        
        self.projection = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )

    def forward(self, x: dict, return_embedding=False):
        batch_size = next(iter(v for k, v in x.items() if k != 'index')).size(0)
        
        img_feat = self.image_encoder(x['image']) if 'image' in x else torch.zeros(batch_size, 128).to(device)
        text_feat = self.text_encoder(x['text']) if 'text' in x else torch.zeros(batch_size, 128).to(device)
        vec_feat = self.vector_encoder(x['vector']) if 'vector' in x else torch.zeros(batch_size, 128).to(device)
        
        img_feat_expand = img_feat.unsqueeze(1)
        text_feat_expand = text_feat.unsqueeze(1)
        vec_feat_expand = vec_feat.unsqueeze(1)
        
        text_attended, _ = self.cross_attn(text_feat_expand, img_feat_expand, img_feat_expand)
        vec_attended, _ = self.cross_attn(vec_feat_expand, img_feat_expand, img_feat_expand)
        
        weights = F.softmax(self.modal_weight, dim=0)
        img_feat_weighted = img_feat * weights[0]
        text_feat_weighted = text_attended.squeeze(1) * weights[1]
        vec_feat_weighted = vec_attended.squeeze(1) * weights[2]
        
        fused = torch.cat([img_feat_weighted, text_feat_weighted, vec_feat_weighted], dim=1)
        unified_embedding = F.relu(self.fusion(fused))  # 统一表征（特征）
        output = self.regressor(unified_embedding)
        projection = self.projection(unified_embedding)
        
        if return_embedding:
            return output, projection, unified_embedding
        return output, projection


# 4. 联邦学习客户端（支持训练集+测试集特征提取，含标签）
class Client:
    def __init__(self, client_id, model, train_dataset, test_dataset=None, learning_rate=0.001):
        self.client_id = client_id
        self.model = copy.deepcopy(model).to(device)
        self.train_dataset = train_dataset  # 本地训练集
        self.test_dataset = test_dataset    # 本地测试集
        self.train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        self.train_feature_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)  # 提取特征用（不打乱）
        self.test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False) if test_dataset else None
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        self.local_epochs = 3
        
        # 互信息映射函数（保持不变）
        self.f_map = nn.Sequential(
            nn.Linear(model.latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        ).to(device)
        for param in self.f_map.parameters():
            param.requires_grad = False

    # 损失函数（保持不变）
    def compute_mi_loss_Y_Z(self, Z, Y):
        f_Z = self.f_map(Z)
        f_Z_norm = (f_Z - f_Z.mean(dim=0)) / (f_Z.std(dim=0) + 1e-8)
        Y_norm = (Y - Y.mean(dim=0)) / (Y.std(dim=0) + 1e-8)
        cov = torch.cov(torch.cat([f_Z_norm, Y_norm], dim=1).T)[0, 1]
        corr = cov / (torch.std(f_Z_norm) * torch.std(Y_norm) + 1e-8)
        return -torch.log(torch.abs(corr) + 1e-8)

    def compute_kl_loss_modal(self, modal_feats):
        if len(modal_feats) < 2:
            return 0.0
        kl_total = 0.0
        num_pairs = 0
        var = nn.Parameter(torch.tensor(0.1)).to(device)
        dist_list = [torch.distributions.Normal(feat, var) for feat in modal_feats]
        
        for i in range(len(dist_list)):
            for j in range(i+1, len(dist_list)):
                kl_ij = torch.distributions.kl.kl_divergence(dist_list[i], dist_list[j]).mean()
                kl_ji = torch.distributions.kl.kl_divergence(dist_list[j], dist_list[i]).mean()
                kl_total += (kl_ij + kl_ji) / 2
                num_pairs += 1
        return kl_total / num_pairs

    def compute_contrastive_loss_anti_forget(self, current_Z, prev_global_Z, history_global_Zs, temperature=0.5):
        current_Z_norm = F.normalize(current_Z, dim=1)
        prev_global_Z_norm = F.normalize(prev_global_Z, dim=1)
        
        pos_samples = prev_global_Z_norm
        if history_global_Zs:
            neg_samples = torch.cat([F.normalize(emb, dim=1) for emb in history_global_Zs], dim=0)
        else:
            neg_samples = current_Z_norm[torch.randperm(current_Z_norm.size(0))]
        
        pos_sim = torch.matmul(current_Z_norm, pos_samples.T).diag() / temperature
        neg_sim = torch.matmul(current_Z_norm, neg_samples.T) / temperature
        logits = torch.cat([pos_sim.unsqueeze(1), neg_sim], dim=1)
        labels = torch.zeros(logits.size(0), dtype=torch.long).to(device)
        return F.cross_entropy(logits, labels)

    def local_train(self, global_model, prev_global_Z, history_global_Zs, mu=0.01):
        """本地训练"""
        self.model.load_state_dict(global_model.state_dict())
        self.model.train()
        total_loss = 0.0
        
        for epoch in range(self.local_epochs):
            for batch in self.train_dataloader:
                current_pred, _, current_Z = self.model(batch, return_embedding=True)
                
                modal_feats = []
                if 'image' in batch:
                    modal_feats.append(self.model.image_encoder(batch['image']))
                if 'text' in batch:
                    modal_feats.append(self.model.text_encoder(batch['text']))
                if 'vector' in batch:
                    modal_feats.append(self.model.vector_encoder(batch['vector']))
                
                reg_loss = F.mse_loss(current_pred.squeeze(), batch['target'])
                mi_loss = self.compute_mi_loss_Y_Z(current_Z, batch['target'].unsqueeze(1))
                kl_loss = self.compute_kl_loss_modal(modal_feats)
                contrast_loss = self.compute_contrastive_loss_anti_forget(
                    current_Z, prev_global_Z, history_global_Zs
                )
                
                loss = reg_loss + mu * (mi_loss + kl_loss + contrast_loss)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()
        
        avg_loss = total_loss / (self.local_epochs * len(self.train_dataloader))
        print(f"客户端 {self.client_id} 平均损失: {avg_loss:.4f}")
        return self.model.state_dict()

    def extract_train_features(self, use_global_model=False, global_model=None):
        """提取训练集特征，与对应标签（y）一起返回"""
        # 选择模型（全局或本地）
        model = global_model if (use_global_model and global_model) else self.model
        model.eval()
        all_features = []
        all_indices = []  # 训练样本索引
        all_targets = []  # 训练样本标签（y）
        
        with torch.no_grad():
            for batch in self.train_feature_dataloader:
                _, _, emb = model(batch, return_embedding=True)
                all_features.append(emb.cpu())
                all_indices.extend(batch['index'].cpu().numpy())
                all_targets.extend(batch['target'].cpu().numpy())  # 收集标签
        
        model.train()
        # 合并结果
        features_np = torch.cat(all_features, dim=0).numpy()
        targets_np = np.array(all_targets)
        
        return {
            'client_id': self.client_id,
            'train_indices': all_indices,
            'train_features': features_np,  # 训练集特征 (N, 128)
            'train_targets': targets_np     # 对应标签 (N,)
        }

    def extract_test_features(self, use_global_model=False, global_model=None):
        """提取测试集特征"""
        if not self.test_dataset:
            return None
        
        model = global_model if (use_global_model and global_model) else self.model
        model.eval()
        all_features = []
        all_indices = []
        all_preds = []
        all_targets = []  # 测试集标签（如有）
        
        with torch.no_grad():
            for batch in self.test_dataloader:
                pred, _, emb = model(batch, return_embedding=True)
                all_features.append(emb.cpu())
                all_indices.extend(batch['index'].cpu().numpy())
                all_preds.append(pred.squeeze().cpu().numpy())
                if 'target' in batch:
                    all_targets.extend(batch['target'].cpu().numpy())  # 测试集标签
        
        model.train()
        features_np = torch.cat(all_features, dim=0).numpy()
        preds_np = np.concatenate(all_preds, axis=0)
        targets_np = np.array(all_targets) if all_targets else None
        
        return {
            'client_id': self.client_id,
            'test_indices': all_indices,
            'test_features': features_np,  # 测试集特征 (N, 128)
            'test_preds': preds_np,         # 预测值 (N,)
            'test_targets': targets_np      # 测试集标签（如有）
        }


# 5. 联邦学习服务器（协调训练集+测试集特征提取）
class Server:
    def __init__(self, model, num_clients):
        self.global_model = copy.deepcopy(model).to(device)
        self.num_clients = num_clients
        self.clients = []
        self.history_global_embs = []
        self.latent_dim = model.latent_dim

    def add_client(self, client):
        self.clients.append(client)

    def aggregate_parameters(self, client_params_list):
        """参数聚合（保持不变）"""
        aggregated_params = {
            name: torch.zeros_like(param) 
            for name, param in self.global_model.state_dict().items()
        }
        
        for params in client_params_list:
            for name, param in params.items():
                if 'num_batches_tracked' in name:
                    aggregated_params[name] += param.long() // self.num_clients
                else:
                    aggregated_params[name] += param.to(aggregated_params[name].dtype) / self.num_clients
        
        self.global_model.load_state_dict(aggregated_params)
        return self.global_model.state_dict()

    def evaluate(self, test_dataset):
        """全局模型评估（保持不变）"""
        if test_dataset is None:
            return
        self.global_model.eval()
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
        total_mse = 0.0
        total_samples = 0
        
        with torch.no_grad():
            for batch in test_loader:
                pred, _ = self.global_model(batch)
                mse = F.mse_loss(pred.squeeze(), batch['target'], reduction='sum')
                total_mse += mse.item()
                total_samples += batch['target'].size(0)
        
        rmse = np.sqrt(total_mse / total_samples)
        #print(f"全局模型测试RMSE: {rmse:.4f}")
        self.global_model.train()
        return rmse

    def federated_train(self, rounds=5, global_test_dataset=None):
        """联邦训练主流程"""
        for round_idx in range(rounds):
            print(f"\n===== 联邦轮次 {round_idx + 1}/{rounds} =====")
            
            client_params_list = []
            
            # 获取上一轮全局表征
            if global_test_dataset and self.history_global_embs:
                test_loader = DataLoader(global_test_dataset, batch_size=32, shuffle=False)
                test_batch = next(iter(test_loader))
                with torch.no_grad():
                    _, _, prev_global_Z = self.global_model(test_batch, return_embedding=True)
                prev_global_Z = prev_global_Z.detach()
                history_global_Zs = [z.detach() for z in self.history_global_embs[-3:]]
            else:
                prev_global_Z = torch.zeros(32, self.latent_dim).to(device)
                history_global_Zs = []
            
            # 客户端本地训练
            for client in self.clients:
                client_params = client.local_train(
                    self.global_model, prev_global_Z, history_global_Zs
                )
                client_params_list.append(client_params)
            
            # 聚合参数
            self.aggregate_parameters(client_params_list)
            
            # 评估
            self.evaluate(global_test_dataset)
        
        return self.global_model

    def get_all_client_features(self, use_global_model=True):
        """获取所有客户端的训练集+测试集特征（含标签）"""
        all_train_features = []
        all_test_features = []
        
        for client in self.clients:
            # 提取训练集特征（含标签）
            train_feats = client.extract_train_features(
                use_global_model=use_global_model,
                global_model=self.global_model if use_global_model else None
            )
            all_train_features.append(train_feats)
            
            # 提取测试集特征
            test_feats = client.extract_test_features(
                use_global_model=use_global_model,
                global_model=self.global_model if use_global_model else None
            )
            if test_feats:
                all_test_features.append(test_feats)
        
        return all_train_features, all_test_features

w:\Anaconda3\envs\DeepSumMoon\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


使用设备: cpu


In [2]:
def generate_sim_data(num_samples=1000):
    image_data = np.random.randn(num_samples, 3, 32, 32)#xpd矩阵
    text_data = np.random.randn(num_samples, 10, 50)
    vector_data = np.random.randn(num_samples, 32)
    img=image_data 
    txt=text_data
    vec=vector_data
    targets =8* 2 / (
            np.exp(
                0.02 * np.sum(img.reshape(img.shape[0], -1)[:, ::10], axis=1)
                + 0.02 * np.sum(txt.reshape(txt.shape[0], -1)[:, ::10], axis=1)
                + 0.02 * np.sum(vec[:, ::10], axis=1)
            ) + np.exp(
                - (
                    0.02 * np.sum(img.reshape(img.shape[0], -1)[:, ::10], axis=1)
                    + 0.02 * np.sum(txt.reshape(txt.shape[0], -1)[:, ::10], axis=1)
                    + 0.02 * np.sum(vec[:, ::10], axis=1))))
    return image_data, text_data, vector_data, targets

# 生成数据
train_image, train_text, train_vector, train_target = generate_sim_data(2000)
global_test_image, global_test_text, global_test_vector, global_test_target = generate_sim_data(500)

# 分配客户端数据
import numpy as np  # 需导入numpy用于生成随机索引

# 分配客户端数据
client_train_datasets = []
client_test_datasets = []

# ---------------------- 训练集随机分配 ----------------------
# 计算训练集总样本数
n_train = len(train_image)  # 原代码中总训练样本为2000（可自动适配实际长度）
# 计算每个客户端的训练样本数（尽量平均分配，余数补到最后一个客户端）
client_train_sizes = [n_train // 3] * 3
remaining_train = n_train % 3
if remaining_train > 0:
    client_train_sizes[-1] += remaining_train  # 余数加到最后一个客户端

# 生成训练集的随机索引（打乱顺序后切分）
train_indices = np.random.permutation(n_train)  # 生成0~n_train-1的随机排列
current_train = 0
client_train_indices = []
for size in client_train_sizes:
    # 切分随机索引，每个客户端对应一组不重叠的索引
    client_train_indices.append(train_indices[current_train:current_train + size])
    current_train += size

# ---------------------- 测试集随机分配 ----------------------
# 计算测试集总样本数
n_test = len(global_test_image)  # 原代码中总测试样本为500（可自动适配实际长度）
# 计算每个客户端的测试样本数（逻辑同训练集）
client_test_sizes = [n_test // 3] * 3
remaining_test = n_test % 3
if remaining_test > 0:
    client_test_sizes[-1] += remaining_test

# 生成测试集的随机索引（打乱顺序后切分）
test_indices = np.random.permutation(n_test)  # 生成0~n_test-1的随机排列
current_test = 0
client_test_indices = []
for size in client_test_sizes:
    client_test_indices.append(test_indices[current_test:current_test + size])
    current_test += size

# ---------------------- 为每个客户端分配数据 ----------------------
for i in range(3):
    # 训练集：使用当前客户端的随机索引
    train_idx = client_train_indices[i]
    train_data = MultimodalDataset(
        image_data=train_image[train_idx],  # 按随机索引取数据
        text_data=train_text[train_idx],
        vector_data=train_vector[train_idx],
        targets=train_target[train_idx],
        is_test=False
    )
    client_train_datasets.append(train_data)

    # 测试集：使用当前客户端的随机索引
    test_idx = client_test_indices[i]
    test_data = MultimodalDataset(
        image_data=global_test_image[test_idx],
        text_data=global_test_text[test_idx],
        vector_data=global_test_vector[test_idx],
        targets=global_test_target[test_idx],
        is_test=True
    )
    client_test_datasets.append(test_data)
    print(f"客户端 {i+1} - 训练样本: {len(train_data)}, 测试样本: {len(test_data)}")

# 全局测试集保持不变
global_test_dataset = MultimodalDataset(
    image_data=global_test_image,
    text_data=global_test_text,
    vector_data=global_test_vector,
    targets=global_test_target,
    is_test=True
)

客户端 1 - 训练样本: 666, 测试样本: 166
客户端 2 - 训练样本: 666, 测试样本: 166
客户端 3 - 训练样本: 668, 测试样本: 168


In [3]:
# 初始化模型和联邦系统
img_encoder = ImageEncoder(output_dim=128)
text_encoder = TextEncoder(input_dim=50, output_dim=128)
vec_encoder = VectorEncoder(input_dim=32, output_dim=128)
base_model = MultimodalEncoder(img_encoder, text_encoder, vec_encoder, latent_dim=128, output_dim=1)

server = Server(base_model, num_clients=3)
for i in range(3):
    client = Client(
        client_id=i+1,
        model=base_model,
        train_dataset=client_train_datasets[i],
        test_dataset=client_test_datasets[i]
    )
    server.add_client(client)

# 联邦训练
print("\n开始联邦训练...")
trained_global_model = server.federated_train(rounds=5, global_test_dataset=global_test_dataset)

# 提取所有客户端的训练集+测试集特征（使用全局模型）
print("\n提取客户端特征...")
all_train_feats, all_test_feats = server.get_all_client_features(use_global_model=True)


开始联邦训练...

===== 联邦轮次 1/5 =====
客户端 1 平均损失: 13.8021
客户端 2 平均损失: 13.6297
客户端 3 平均损失: 13.5192

===== 联邦轮次 2/5 =====
客户端 1 平均损失: 1.0998
客户端 2 平均损失: 1.2401
客户端 3 平均损失: 1.2195

===== 联邦轮次 3/5 =====
客户端 1 平均损失: 0.9299
客户端 2 平均损失: 1.0363
客户端 3 平均损失: 1.1272

===== 联邦轮次 4/5 =====
客户端 1 平均损失: 0.8900
客户端 2 平均损失: 0.9812
客户端 3 平均损失: 0.8628

===== 联邦轮次 5/5 =====
客户端 1 平均损失: 0.7850
客户端 2 平均损失: 0.8428
客户端 3 平均损失: 0.7433

提取客户端特征...


In [4]:
# 输出训练集特征与标签
print("\n===== 训练集特征与标签 =====")
for tf in all_train_feats:
    client_id = tf['client_id']
    print(f"客户端 {client_id} 训练集:")
    print(f"  样本数: {len(tf['train_indices'])}")
    print(f"  特征形状: {tf['train_features'].shape}")
    #print(f"  第1个样本特征前5维: {tf['train_features'][0, :5].round(4)}")
    #print(f"  第1个样本标签: {tf['train_targets'][0].round(4)}\n")  # 训练集标签

# 输出测试集特征
print("===== 测试集特征与预测 =====")
for tf in all_test_feats:
    client_id = tf['client_id']
    print(f"客户端 {client_id} 测试集:")
    print(f"  样本数: {len(tf['test_indices'])}")
    print(f"  特征形状: {tf['test_features'].shape}")
    #print(f"  第1个样本特征前5维: {tf['test_features'][0, :5].round(4)}")
    #print(f"  第1个样本预测值: {tf['test_preds'][0].round(4)}")
    #print(f"  第1个样本真实标签: {tf['test_targets'][0].round(4)}\n")


===== 训练集特征与标签 =====
客户端 1 训练集:
  样本数: 666
  特征形状: (666, 128)
客户端 2 训练集:
  样本数: 666
  特征形状: (666, 128)
客户端 3 训练集:
  样本数: 668
  特征形状: (668, 128)
===== 测试集特征与预测 =====
客户端 1 测试集:
  样本数: 166
  特征形状: (166, 128)
客户端 2 测试集:
  样本数: 166
  特征形状: (166, 128)
客户端 3 测试集:
  样本数: 168
  特征形状: (168, 128)


In [5]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

# 存储各客户端的评估结果
results = []

# 遍历每个客户端进行模型训练与评估
print("\n===== 模型训练与评估 =====")
for i in range(len(all_train_feats)):
    # 获取当前客户端的训练数据和测试数据
    train_data = all_train_feats[i]
    test_data = all_test_feats[i]
    client_id = train_data['client_id']
    
    print(f"处理客户端 {client_id}...")
    
    # 提取训练特征和标签
    X_train = train_data['train_features']
    y_train = train_data['train_targets']
    
    # 提取测试特征和真实标签
    X_test = test_data['test_features']
    y_test = test_data['test_targets']
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    # 初始化并训练随机森林模型
    rf_model =RandomForestRegressor(
        n_estimators=100,
        n_jobs=-1
    )
    rf_model.fit(X_train, y_train)
    
    # 在测试集上进行预测
    y_pred = rf_model.predict(X_test)
    
    # 计算均方误差
    mse = mean_squared_error(y_test, y_pred)
    
    # 存储结果
    results.append({
        '客户端ID': client_id,
        '训练样本数': len(train_data['train_indices']),
        '测试样本数': len(test_data['test_indices']),
        '特征维度': X_train.shape[1],
        '均方误差(MSE)': round(mse, 6)
    })
    
    # 保存预测结果到测试数据中（如果需要）
    test_data['test_preds'] = y_pred

# 汇总结果为表格
results_df1 = pd.DataFrame(results)

# 输出汇总表格
print("\n===== 各客户端模型评估结果汇总 =====")
print(results_df1)

# 可选：将结果保存为CSV文件
# results_df1.to_csv('client_model_results.csv', index=False)


===== 模型训练与评估 =====
处理客户端 1...
处理客户端 2...
处理客户端 3...

===== 各客户端模型评估结果汇总 =====
   客户端ID  训练样本数  测试样本数  特征维度  均方误差(MSE)
0      1    666    166   128   0.408163
1      2    666    166   128   0.403993
2      3    668    168   128   0.364085


In [6]:
# ==== 基线对比：仅 VAE + 您的方法（回填） ====
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# -------------------------------
# 工具：拼接多模态 → 二维特征
# -------------------------------
def flatten_and_concat(image, text, vector):
    Xi = image.reshape(image.shape[0], -1)   # e.g. 3*32*32
    Xt = text.reshape(text.shape[0], -1)     # e.g. 10*50
    Xv = vector                              # e.g. 32
    return np.hstack([Xi, Xt, Xv])

# -------------------------------
# 最简 VAE（β-VAE 可用 beta>1）
# -------------------------------
class VAE(nn.Module):
    def __init__(self, in_dim, z_dim=16):
        super().__init__()
        self.enc = nn.Sequential(
            nn.Linear(in_dim, 128), nn.ReLU(),
            nn.Linear(128, 64), nn.ReLU()
        )
        self.mu = nn.Linear(64, z_dim)
        self.logvar = nn.Linear(64, z_dim)
        self.dec = nn.Sequential(
            nn.Linear(z_dim, 64), nn.ReLU(),
            nn.Linear(64, 128), nn.ReLU(),
            nn.Linear(128, in_dim)
        )

    def encode(self, x):
        h = self.enc(x)
        return self.mu(h), self.logvar(h)

    def reparam(self, mu, logvar):
        std = (0.5 * logvar).exp()
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparam(mu, logvar)
        x_hat = self.dec(z)
        return x_hat, mu, logvar, z

def vae_loss(x, x_hat, mu, logvar, beta=1.0):
    recon = F.mse_loss(x_hat, x, reduction='mean')
    kld = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return recon + beta * kld

# -------------------------------
# 训练 VAE -> 抽取潜变量 z -> RF 回归
# -------------------------------
def train_rf_with_vae(train_data, test_data,
                      z_dim=16, beta=1.0, epochs=80, batch_size=256, lr=1e-3,
                      n_estimators=5, random_state=0):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.manual_seed(random_state)
    np.random.seed(random_state)

    # 1) 组装特征并标准化（按训练集拟合）
    X_tr_raw = flatten_and_concat(train_data.image_data, train_data.text_data, train_data.vector_data)
    X_te_raw = flatten_and_concat(test_data.image_data,  test_data.text_data,  test_data.vector_data)
    scaler = StandardScaler().fit(X_tr_raw)
    X_tr = scaler.transform(X_tr_raw).astype(np.float32)
    X_te = scaler.transform(X_te_raw).astype(np.float32)

    # 2) 训练 VAE（仅用训练集）
    vae = VAE(in_dim=X_tr.shape[1], z_dim=z_dim).to(device)
    opt = torch.optim.Adam(vae.parameters(), lr=lr)
    dl = DataLoader(TensorDataset(torch.from_numpy(X_tr)), batch_size=batch_size, shuffle=True)

    vae.train()
    for _ in range(epochs):
        for (xb,) in dl:
            xb = xb.to(device)
            xh, mu, logv, _ = vae(xb)
            loss = vae_loss(xb, xh, mu, logv, beta)
            opt.zero_grad(); loss.backward(); opt.step()

    # 3) 抽取潜变量（使用 mu 作为稳定特征）
    vae.eval()
    with torch.no_grad():
        Ztr = []
        for (xb,) in DataLoader(TensorDataset(torch.from_numpy(X_tr)), batch_size=4096, shuffle=False):
            xb = xb.to(device)
            _, mu, _, _ = vae(xb)
            Ztr.append(mu.cpu())
        Ztr = torch.cat(Ztr, dim=0).numpy()

        Zte = []
        for (xb,) in DataLoader(TensorDataset(torch.from_numpy(X_te)), batch_size=4096, shuffle=False):
            xb = xb.to(device)
            _, mu, _, _ = vae(xb)
            Zte.append(mu.cpu())
        Zte = torch.cat(Zte, dim=0).numpy()

    # 4) 用 RF 做回归并评估
    y_train, y_test = train_data.targets, test_data.targets
    rf = RandomForestRegressor(n_estimators=n_estimators, max_features='auto',
                               n_jobs=-1, random_state=random_state)
    rf.fit(Ztr, y_train)
    y_pred = rf.predict(Zte)
    mse = mean_squared_error(y_test, y_pred)

    return mse, {"z_dim": z_dim}

# -------------------------------
# 主流程（只输出 VAE 与 您的方法 占位）
# -------------------------------
comparison_rows = []
print("===== 基线对比：仅 VAE（+ 您的方法 占位）=====")
for client_idx in range(3):
    client_id = client_idx + 1
    train_data = client_train_datasets[client_idx]
    test_data  = client_test_datasets[client_idx]

    vae_mse, vae_info = train_rf_with_vae(
        train_data, test_data,
        z_dim=16, beta=1.0, epochs=80, batch_size=256, lr=1e-3,
        n_estimators=5, random_state=0
    )

    row = {
        '客户端ID': client_id,
        '训练样本数': len(train_data),
        '测试样本数': len(test_data),
        'VAE潜维度': vae_info['z_dim'],
        'VAE降维RF-MSE': round(float(vae_mse), 6),
        '您的方法-MSE': None  # 由你外部结果回填
    }
    comparison_rows.append(row)

results_df = pd.DataFrame(comparison_rows)

# 若你已有 “您的方法” 的 MSE 序列，可在此回填（例如来自 results_df1 的最后一列）
try:
    your_method_mses = results_df1.iloc[:, -1]
    for i in range(len(results_df)):
        results_df.loc[i, '您的方法-MSE'] = round(float(your_method_mses[i]), 6)
except Exception:
    pass  # 没有就留空，后续再回填

print("\n===== 两方法对比（仅你要求的列） =====")
results_df


===== 基线对比：仅 VAE（+ 您的方法 占位）=====

===== 两方法对比（仅你要求的列） =====


,客户端ID,训练样本数,测试样本数,VAE潜维度,VAE降维RF-MSE,您的方法-MSE
0,1,666,166,16,0.770507,0.408163
1,2,666,166,16,0.792421,0.403993
2,3,668,168,16,1.065883,0.364085
